In [1]:
import pandas as pd

# Load data dari folder raw/
users = pd.read_csv('users_clean.csv')
tracking = pd.read_csv('tracking_clean.csv')
submission = pd.read_csv('submission_clean.csv')
complete = pd.read_csv('complete_clean.csv')
registration = pd.read_csv('registration_clean.csv')
exam = pd.read_csv('exam_clean.csv')

# Cek struktur awal
print("Users:", users.shape)
print("Tracking:", tracking.shape)
print("Submission:", submission.shape)
print("Complete:", complete.shape)
print("Registration:", registration.shape)
print("Exam:", exam.shape)



Users: (31, 10)
Tracking: (101736, 7)
Submission: (2262, 19)
Complete: (1032, 10)
Registration: (16759, 11)
Exam: (17438, 7)


In [2]:
# 1️⃣ Base utama dari tracking (aktivitas belajar)
df = tracking.copy()

# 2️⃣ Gabung dengan submission (nilai tugas)
df = pd.merge(
    df, submission,
    left_on=['developer_id', 'journey_id'],
    right_on=['submitter_id', 'journey_id'],
    how='left'
)

# 3️⃣ Gabung dengan complete (hasil journey)
df = pd.merge(
    df, complete,
    left_on=['developer_id', 'journey_id'],
    right_on=['user_id', 'journey_id'],
    how='left'
)

# 4️⃣ Gabung registration + exam jadi satu dulu
exam_full = pd.merge(
    registration, exam,
    left_on='id',
    right_on='exam_registration_id',
    how='left'
)

# 5️⃣ Gabungkan hasil ujian ke data utama
df = pd.merge(
    df, exam_full,
    left_on=['developer_id', 'journey_id'],
    right_on=['examinees_id', 'tutorial_id'],
    how='left'
)

# 6️⃣ Tambahkan identitas user (aman dari duplikat kolom)
users_cleaned = users.copy()

# Drop kolom yang sering bentrok
dupes = [c for c in users_cleaned.columns if c in df.columns]
if dupes:
    users_cleaned = users_cleaned.drop(columns=dupes)

df = pd.merge(
    df,
    users_cleaned,
    left_on='developer_id',
    right_on='id',
    how='left',
    suffixes=('', '_user')
)

df.to_csv('merged_data.csv', index=False)
print("✅ merged_data.csv created:", df.shape)



✅ merged_data.csv created: (292493, 61)


In [3]:
drop_cols = [c for c in df.columns if any(x in c.lower() for x in ['applink', 'appcomment', 'note'])]
if drop_cols:
    df = df.drop(columns=drop_cols)
    print(f"🧽 Kolom berikut dihapus karena tidak diperlukan: {drop_cols}")
else:
    print("ℹ️ Tidak ditemukan kolom aplink, appcomment, atau note.")

🧽 Kolom berikut dihapus karena tidak diperlukan: ['note']


In [4]:
print("\n🧹 Mulai cleaning dan transformasi...")

# Konversi kolom waktu ke datetime
datetime_cols = [
    'first_opened_at', 'completed_at', 'last_viewed',
    'created_at_x', 'updated_at_x', 'created_at_y', 'updated_at_y'
]
for col in datetime_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Hapus duplikat berdasarkan kombinasi unik
expected_cols = ['developer_id', 'journey_id', 'tutorial_id']
available_cols = [c for c in expected_cols if c in df.columns]
if len(available_cols) > 0:
    df = df.drop_duplicates(subset=available_cols, keep='first')
    print(f"✅ Duplikat dihapus berdasarkan: {available_cols}")
else:
    print("⚠️ Tidak ada kolom dari daftar kunci duplikat ditemukan!")

# Hitung durasi belajar per materi (dalam menit)
if 'completed_at' in df.columns and 'first_opened_at' in df.columns:
    df['learning_duration_min'] = (
        (df['completed_at'] - df['first_opened_at']).dt.total_seconds() / 60
    )
    df['learning_duration_min'] = df['learning_duration_min'].fillna(0).clip(lower=0)
    print("✅ learning_duration_min berhasil dihitung!")
else:
    print("⚠️ Kolom waktu tidak lengkap, durasi belajar tidak dihitung.")


🧹 Mulai cleaning dan transformasi...
✅ Duplikat dihapus berdasarkan: ['developer_id', 'journey_id']
✅ learning_duration_min berhasil dihitung!


In [6]:
df.to_csv('merged_data.csv', index=False)
print("\n🎉 Cleaning selesai! Hasil akhir disimpan di: data/processed/merged_data.csv")
print("Ukuran akhir dataset:", df.shape)


🎉 Cleaning selesai! Hasil akhir disimpan di: data/processed/merged_data.csv
Ukuran akhir dataset: (2013, 61)


In [7]:
drop_cols = [
    'id_x_x', 'id_y_x', 'id_x_y', 'id_y_y',
    'quiz_id', 'submitter_id', 'version_id',
    'app_link', 'app_comment', 'admin_comment',
    'status_y', 'as_trial_subscriber', 'current_reviewer',
    'started_review_at', 'ended_review_at',
    'submission_duration', 'created_at_x_x', 'updated_at_x',
    'created_at_y_x', 'updated_at_y',
    'enrollments_at', 'last_enrolled_at',
    'deleted_at', 'deadline_at', 'retake_limit_at',
    'exam_finished_at', 'look_report_at',
    'exam_module_id', 'tutorial_id_y', 'examinees_id',
    'status', 'created_at_x_y', 'created_at_y_y', 'updated_at',
    'id', 'name', 'phone', 'user_verification_status'
]


In [8]:
# Hapus kolom tidak penting
drop_cols = [c for c in drop_cols if c in df.columns]
df = df.drop(columns=drop_cols)

print(f"🧽 {len(drop_cols)} kolom dihapus: {drop_cols}")


🧽 38 kolom dihapus: ['id_x_x', 'id_y_x', 'id_x_y', 'id_y_y', 'quiz_id', 'submitter_id', 'version_id', 'app_link', 'app_comment', 'admin_comment', 'status_y', 'as_trial_subscriber', 'current_reviewer', 'started_review_at', 'ended_review_at', 'submission_duration', 'created_at_x_x', 'updated_at_x', 'created_at_y_x', 'updated_at_y', 'enrollments_at', 'last_enrolled_at', 'deleted_at', 'deadline_at', 'retake_limit_at', 'exam_finished_at', 'look_report_at', 'exam_module_id', 'tutorial_id_y', 'examinees_id', 'status', 'created_at_x_y', 'created_at_y_y', 'updated_at', 'id', 'name', 'phone', 'user_verification_status']


In [9]:
df.rename(columns={
    'tutorial_id_x': 'tutorial_id',
    'status_x': 'status'
}, inplace=True)


In [10]:
if 'user_id' in df.columns:
    if df['developer_id'].equals(df['user_id']):
        print("✅ developer_id dan user_id identik, user_id dihapus.")
        df.drop(columns=['user_id'], inplace=True)
    else:
        print("⚠️ Nilai berbeda, jangan hapus dulu! Periksa manual.")


⚠️ Nilai berbeda, jangan hapus dulu! Periksa manual.


In [11]:
mask_diff = df['developer_id'] != df['user_id']
diff_rows = df.loc[mask_diff, ['developer_id', 'user_id', 'journey_id']]

print(f"🔎 Jumlah baris berbeda: {mask_diff.sum()}")
print("Beberapa contoh perbedaan:")
print(diff_rows.head(10))


🔎 Jumlah baris berbeda: 981
Beberapa contoh perbedaan:
      developer_id  user_id  journey_id
0            96989      NaN          26
8            96989      NaN          72
106          96989      NaN          92
118          96989      NaN          14
126          96989      NaN          60
421          96989      NaN          80
442          96989      NaN         135
1191         96989      NaN         116
1306         96989      NaN         202
1582         96989      NaN          65


In [12]:
# Isi user_id yang kosong dengan developer_id
df['user_id'] = df['user_id'].fillna(df['developer_id'])
print("🧩 user_id yang kosong diisi dengan developer_id.")

# Hapus kolom user_id karena sudah redundant
df.drop(columns=['user_id'], inplace=True)
print("✅ Kolom user_id dihapus, developer_id tetap dipakai sebagai ID utama.")


🧩 user_id yang kosong diisi dengan developer_id.
✅ Kolom user_id dihapus, developer_id tetap dipakai sebagai ID utama.


In [14]:
df.to_csv('merged_cleaned.csv', index=False)
print("✅ merged_cleaned.csv diperbarui dengan hasil cleaning (user_id dihapus)")


✅ merged_cleaned.csv diperbarui dengan hasil cleaning (user_id dihapus)


In [16]:
import pandas as pd

df = pd.read_csv("merged_cleaned.csv")
print(df[['developer_id', 'first_opened_at', 'completed_at']].head(10))


   developer_id      first_opened_at         completed_at
0         96989                  NaN                  NaN
1         96989  2018-08-29 11:19:00                  NaN
2         96989  2018-09-02 17:11:00                  NaN
3         96989  2018-10-04 23:09:00                  NaN
4         96989  2018-11-29 11:38:00                  NaN
5         96989  2018-12-13 16:39:00                  NaN
6         96989  2018-12-15 10:28:00                  NaN
7         96989  2019-05-10 14:24:00                  NaN
8         96989  2019-05-18 20:06:00  2019-05-18 20:10:00
9         96989  2019-06-11 12:44:00                  NaN


In [17]:
# Konversi kolom waktu ke datetime
df['first_opened_at'] = pd.to_datetime(df['first_opened_at'], errors='coerce')
df['completed_at'] = pd.to_datetime(df['completed_at'], errors='coerce')

# Cek hasil konversi (5 baris pertama)
print(df[['developer_id', 'first_opened_at', 'completed_at']].head(10))
print("\nTipe data kolom waktu:")
print(df[['first_opened_at', 'completed_at']].dtypes)


   developer_id     first_opened_at        completed_at
0         96989                 NaT                 NaT
1         96989 2018-08-29 11:19:00                 NaT
2         96989 2018-09-02 17:11:00                 NaT
3         96989 2018-10-04 23:09:00                 NaT
4         96989 2018-11-29 11:38:00                 NaT
5         96989 2018-12-13 16:39:00                 NaT
6         96989 2018-12-15 10:28:00                 NaT
7         96989 2019-05-10 14:24:00                 NaT
8         96989 2019-05-18 20:06:00 2019-05-18 20:10:00
9         96989 2019-06-11 12:44:00                 NaT

Tipe data kolom waktu:
first_opened_at    datetime64[ns]
completed_at       datetime64[ns]
dtype: object


In [18]:
# Hitung durasi belajar (dalam menit)
df['learning_duration_min'] = (
    (df['completed_at'] - df['first_opened_at']).dt.total_seconds() / 60
)

# Bersihkan hasil: ganti NaN jadi 0, dan hindari nilai negatif
df['learning_duration_min'] = df['learning_duration_min'].fillna(0).clip(lower=0)

# Lihat contoh hasilnya
print(df[['developer_id', 'first_opened_at', 'completed_at', 'learning_duration_min']].head(15))


    developer_id     first_opened_at        completed_at  \
0          96989                 NaT                 NaT   
1          96989 2018-08-29 11:19:00                 NaT   
2          96989 2018-09-02 17:11:00                 NaT   
3          96989 2018-10-04 23:09:00                 NaT   
4          96989 2018-11-29 11:38:00                 NaT   
5          96989 2018-12-13 16:39:00                 NaT   
6          96989 2018-12-15 10:28:00                 NaT   
7          96989 2019-05-10 14:24:00                 NaT   
8          96989 2019-05-18 20:06:00 2019-05-18 20:10:00   
9          96989 2019-06-11 12:44:00                 NaT   
10         96989 2019-06-14 18:18:00                 NaT   
11         96989                 NaT                 NaT   
12         96989                 NaT                 NaT   
13         96989                 NaT                 NaT   
14         96989                 NaT                 NaT   

    learning_duration_min  
0          

In [19]:
# Hitung total durasi belajar per user (dalam menit)
study_duration_total = (
    df.groupby('developer_id')['learning_duration_min']
      .sum()
      .reset_index()
      .rename(columns={'learning_duration_min': 'study_duration_total'})
)

# Urutkan biar rapi (opsional)
study_duration_total = study_duration_total.sort_values(by='study_duration_total', ascending=False)

# Lihat contoh hasil
print("📊 Total durasi belajar per mentee (dalam menit):")
print(study_duration_total.head(10))


📊 Total durasi belajar per mentee (dalam menit):
    developer_id  study_duration_total
6         102556             2507055.0
10        433737             2435922.0
3          17833             1605636.0
4          32258             1425199.0
23       2395575             1010848.0
1           5774              904847.0
21       2131303              743334.0
15        791882              288715.0
11        494342              224197.0
13        518124              102719.0


In [20]:
# Batasi durasi maksimum per sesi menjadi 480 menit (8 jam)
df['learning_duration_min_capped'] = df['learning_duration_min'].clip(upper=480)

# Hitung ulang total durasi belajar
study_duration_total_capped = (
    df.groupby('developer_id')['learning_duration_min_capped']
      .sum()
      .reset_index()
      .rename(columns={'learning_duration_min_capped': 'study_duration_total'})
      .sort_values(by='study_duration_total', ascending=False)
)

print("📊 Total durasi belajar setelah pembatasan (max 8 jam per sesi):")
print(study_duration_total_capped.head(10))


📊 Total durasi belajar setelah pembatasan (max 8 jam per sesi):
    developer_id  study_duration_total
12        496740                2883.0
6         102556                1956.0
4          32258                1924.0
10        433737                1447.0
13        518124                1442.0
21       2131303                1244.0
11        494342                1010.0
22       2135328                 973.0
1           5774                 962.0
0           3390                 961.0


In [21]:
study_duration_total_capped.to_csv("study_duration_recomputed.csv", index=False)
print("✅ study_duration_recomputed.csv berhasil disimpan!")


✅ study_duration_recomputed.csv berhasil disimpan!
